In [10]:
import pymupdf,fitz
import re

In [ ]:
def extract_text_from_pdf(pdf_path, output_txt_path, start_page, end_page):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(start_page - 1, end_page):  
        text += doc[page_num].get_text("text") + "\n\n" 
    with open(output_txt_path, "w", encoding="utf-8") as txt_file:
        txt_file.write(text)

    print(f"Text extracted from pages {start_page} to {end_page} and saved to {output_txt_path}")
    
pdf_file = "PDF_FILES\DIM evaluation report 22 Oct 2024.pdf" 
output_text_file = "probable_toc.txt"
extract_text_from_pdf(pdf_file, output_text_file, 2, 8)

Text extracted from pages 2 to 8 and saved to probable_toc.txt


In [14]:
def find_chapter_pages(toc_text, chapter_name):
    pattern = r'^\s*(\d*\.*\s*)?([A-Z][A-Z\s\-]+)\s*[\.…]*\s*(\d+)\s*$'
    matches = re.findall(pattern, toc_text, re.MULTILINE)

    chapters = []
    for match in matches:
        number, title, page = match
        title = title.strip()
        page = int(page)
        chapters.append((title, page))
    start_page = None
    end_page = None

    for i, (title, page) in enumerate(chapters):
        if title.lower() == chapter_name.lower():
            start_page = page
            if i + 1 < len(chapters):
                end_page = chapters[i + 1][1] - 1  
            break

    return start_page, end_page

with open("probable_toc.txt", "r", encoding="utf-8") as file:
    toc_text = file.read()

chapter_name = "FINDINGS"

start_page, end_page = find_chapter_pages(toc_text, chapter_name)

if start_page and end_page:
    print(f"Chapter '{chapter_name}' starts on page {start_page} and ends on page {end_page}.")
else:
    print(f"Could not find accurate page numbers for '{chapter_name}'.")

Chapter 'FINDINGS' starts on page 44 and ends on page 61.
